<a href="https://colab.research.google.com/github/WM-CSCI-435-F19/Neural-Unsupervised-Software-Traceability/blob/master/notebook/modeling/doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gensim
import os
import collections
import smart_open
import random

In [0]:
# Function to help read a .cor file
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [0]:
# Create a new .cor file
iTrust_cor_file = open("iTrust.cor","w+")

# For testing I uploaded the contents of Neural-Unsupervised-Software-Traceability/data/raw/iTrust_semeru_format/use_cases/
# to the below directory
for filename in os.listdir('/content/github_datasets/iTrust'):
  # get the filepath for the current file and open it
  cur_path = '/content/github_datasets/iTrust/' + filename
  cur_file = open(cur_path, "r", encoding="ISO-8859-1")
  # create a new empty string, read in file and remove newlines
  one_line_str = ""
  for line in cur_file :
    one_line_str += line.strip() + " "
  # add new one-line string to iTrust .cor file and add a newline
  iTrust_cor_file.write(one_line_str + "\n")
  cur_file.close()
iTrust_cor_file.close()
filepath = '/content/iTrust.cor'
# convert our .cor file to a list that can be fed into the model
train_corpus = list(read_corpus(filepath))

In [0]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [0]:
model.build_vocab(train_corpus)

In [29]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 505 ms, sys: 17.2 ms, total: 522 ms
Wall time: 537 ms


In [31]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [32]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (130): «the chosen patient is not the desired patient the lhcp does not confirm the selection and can try again»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (115, 0.9708189368247986): «the patient chosen is not the desired patient the hcp does not confirm the selection and is prompted to try again»

SECOND-MOST (120, 0.9704044461250305): «the patient chosen is not the desired patient the lhcp does not confirm the selection and is prompted to try again»

MEDIAN (97, 0.5207624435424805): «user chooses to view physician satisfaction survey results the user provides zip code or hospital code and an optional physician type from pull down list see data format general surgeon heart specialist pediatrician ob gyn the patient is provided with the following for each physician of that type that practices in zip code based upon the address zipcode provided in uc that match the first three digits of the provided zip code name address average number of mi